**Some Imports**

In [39]:
import numpy as np
import pandas as pd
import nltk
import re

**Malay Stemmer** 
stolen from https://github.com/huseinzol05/Malaya/blob/master/session/emotion/multinomial.ipynb

In [40]:
permulaan = [
    'bel',
    'se',
    'ter',
    'men',
    'meng',
    'mem',
    'memper',
    'di',
    'pe',
    'me',
    'ke',
    'ber',
    'pen',
    'per',
]

hujung = ['kan', 'kah', 'lah', 'tah', 'nya', 'an', 'wan', 'wati', 'ita']

def naive_stemmer(word):
    assert isinstance(word, str), 'input must be a string'
    hujung_result = re.findall(r'^(.*?)(%s)$' % ('|'.join(hujung)), word)
    word = hujung_result[0][0] if len(hujung_result) else word
    permulaan_result = re.findall(r'^(.*?)(%s)' % ('|'.join(permulaan[::-1])), word)
    permulaan_result.extend(re.findall(r'^(.*?)(%s)' % ('|'.join(permulaan)), word))
    mula = permulaan_result if len(permulaan_result) else ''
    if len(mula):
        mula = mula[1][1] if len(mula[1][1]) > len(mula[0][1]) else mula[0][1]
    return word.replace(mula, '')

**File import**
note that I change Sarcasm_Headlines_Dataset.json to Sarcasm_Headlines_Dataset2.json due to some f ups

In [2]:
import os
print(os.listdir("../Sarcasm"))

['.ipynb_checkpoints', 'bm-sarcastic-news-headline.json', 'Sarcasm Detection English.ipynb', 'Sarcasm_Headlines_Dataset - Copy.json', 'Sarcasm_Headlines_Dataset.json', 'Sarcasm_Headlines_Dataset2.json', 'Untitled.ipynb', '__MACOSX']


In [3]:
dataset_en = pd.read_json('../Sarcasm/Sarcasm_Headlines_Dataset2.json',lines=True)
dataset_my = pd.read_json('../Sarcasm/bm-sarcastic-news-headline.json',lines = True)

**Splitting and Rejoining Malay Headlines**

In [37]:
my_sar=pd.DataFrame({'headline':dataset_my.iloc[0,1],'is_sarcastic':1})
my_nsar= pd.DataFrame({'headline':dataset_my.iloc[0,0],'is_sarcastic':0})
data_my = my_sar.append(my_nsar, ignore_index = True)

**Excluding Website in English Headlines**

In [28]:
data_en = dataset_en.iloc[:,1:]

**Imports for Stopwords, Tokenizer and Lemmatizer**

In [31]:
from nltk.corpus import stopwords
import string
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer


**Removing Stopwords, Tokenizer and Lemmatizer for English**

In [78]:
stop_en = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

tokenizer_nltk = RegexpTokenizer('[0-9]*\.[0-9]*|\w+')
for i in range(len(data_en)):
    tokens = tokenizer_nltk.tokenize(data_en.iloc[i,0])
    tokens = [lemmatizer.lemmatize(x)for x in tokens]
    tok = [x for x in tokens if ((x not in stop_en) and (x not in string.punctuation))]
    data_en.iloc[i,0] = ' '.join(tok)
data_en_clean = data_en

**Stopword from Malay Language**

Words obtain from:
https://blog.kerul.net/2014/01/list-of-malay-stop-words.html
List contains words with -lah,-kah etc, which is redundant

In [42]:
stop_my = ['ada','inikah','sampai','adakah','inilah','sana',
        'adakan','itu','sangat','adalah','itukah','sangatlah',
        'adanya','itulah','saya','adapun','jadi','se','agak',
        'jangan','seandainya','agar','janganlah','sebab','akan',
        'jika','sebagai','aku','jikalau','sebagaimana','akulah',
        'jua','sebanyak','akupun','juapun','sebelum','al','juga',
        'sebelummu','alangkah','kalau','sebelumnya','allah',
        'kami','sebenarnya','amat','kamikah','secara','antara',
        'kamipun','sedang','antaramu','kamu','edangkan',
        'antaranya','amukah','edikit','apa','kamupun',
        'sedikitpun','apa-apa','katakan','segala','apabila',
        'ke','sehingga','apakah','kecuali','sejak','apapun',
        'kelak','sekalian','atas','kembali','sekalipun','atasmu',
        'kemudian','sekarang','atasnya','kepada','sekitar',
        'atau','kepadaku','selain','ataukah','kepadakulah',
        'selalu','ataupun','kepadamu','selama','bagaimana',
        'kepadanya','selama-lamanya','bagaimanakah',
        'kepadanyalah','seluruh','bagi','kerana','seluruhnya',
        'bagimu','kerananya','sementara','baginya','kesan',
        'semua','bahawa','ketika','semuanya','bahawasanya',
        'kini','semula','bahkan','kita','senantiasa','bahwa',
        'ku','sendiri','banyak','kurang','sentiasa','banyaknya',
        'lagi','seolah','barangsiapa','lain','seolah-olah',
        'bawah','lalu','seorangpun','beberapa','lamanya',
        'separuh','begitu','langsung','sepatutnya','begitupun',
        'lebih','seperti','belaka','maha','seraya','belum','mahu',
        'sering','belumkah','mahukah','serta','berada',
        'mahupun','seseorang','berapa','maka','sesiapa','berikan',
        'malah','sesuatu','beriman','mana','sesudah','berkenaan',
        'manakah','sesudahnya','berupa','manapun','sesungguhnya',
        'beserta','masih','sesungguhnyakah','biarpun','masing',
        'setelah','bila','masing-masing','setiap','bilakah',
        'melainkan','siapa','bilamana','memang','siapakah',
        'bisa','mempunyai','sini','boleh','mendapat','situ',
        'bukan','mendapati','situlah','bukankah','mendapatkan',
        'suatu','bukanlah','mengadakan','sudah','dahulu',
        'mengapa','sudahkah','dalam','mengapakah','sungguh',
        'dalamnya','mengenai','sungguhpun','dan','menjadi',
        'supaya','dapat','menyebabkan','tadinya','dapati',
        'menyebabkannya','tahukah','dapatkah','mereka','tak',
        'dapatlah','merekalah','tanpa','dari','merekapun',
        'tanya','daripada','meskipun','tanyakanlah','daripadaku',
        'mu','tapi','daripadamu','nescaya','telah','daripadanya',
        'niscaya','tentang','demi','nya','tentu','demikian',
        'olah','terdapat','demikianlah','oleh','terhadap',
        'dengan','orang','terhadapmu','dengannya','pada',
        'termasuk','di','padahal','terpaksa','dia','padamu',
        'tertentu','dialah','padanya','tetapi','didapat',
        'paling','tiada','didapati','para','tiadakah',
        'dimanakah','pasti','tiadalah','engkau','patut',
        'tiap','engkaukah','patutkah','tiap-tiap','engkaulah',
        'per','tidak','engkaupun','pergilah','tidakkah','hai',
        'perkara','tidaklah','hampir','perkaranya','turut',
        'hampir-hampir','perlu','untuk','hanya','pernah',
        'untukmu','anyalah','pertama','wahai','hendak','pula',
        'walau','hendaklah','pun','walaupun','hingga','sahaja',
        'ya','ia','saja','yaini','iaitu','saling','yaitu',
        'ialah','sama','yakni','ianya','sama-sama','yang',
        'inginkah','samakah','ini','sambil']

**Tokenizing, Stemming and Removing StopWords**

In [48]:
tokenizer_nltk = RegexpTokenizer('[0-9]*\.[0-9]*|\w+')
for i in range(len(data_my)):
    tokens = tokenizer_nltk.tokenize(data_my.iloc[i,0])
    tokens = [naive_stemmer(x) for x in tokens]
    tok = [x for x in tokens if ((x not in stop_my) and (x not in string.punctuation))]
    data_my.iloc[i,0] = ' '.join(tok)
data_my_clean = data_my   

In [74]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [79]:
tfidf = TfidfVectorizer(ngram_range=(1,3),min_df=2).fit(data_en_clean['headline'])
en_vectors = tfidf.transform(data_en_clean['headline'])

tfidf = TfidfVectorizer(ngram_range=(1,3),min_df=2).fit(data_my_clean['headline'])
my_vectors = tfidf.transform(data_my_clean['headline'])


**Run this to put Malay Dataset into Test and Training set**

In [70]:
train_X, test_X, train_Y, test_Y = train_test_split(my_vectors, data_my_clean['is_sarcastic'], test_size = 0.2)

**Run this to put English Dataset into Test and Training set**

In [85]:
train_X, test_X, train_Y, test_Y = train_test_split(en_vectors, data_en_clean['is_sarcastic'], test_size = 0.2)

**Fitting Data into model and predicting Test Data**

no idea which model to use, followed https://github.com/huseinzol05/Malaya/blob/master/session/emotion/multinomial.ipynb
model

In [89]:
model  = MultinomialNB().fit(train_X, train_Y)
print(
    metrics.classification_report(
        train_Y,
        model.predict(train_X),
        target_names = ['is_sarcastic','not_sarcastic'],
    )
)

               precision    recall  f1-score   support

 is_sarcastic       0.88      0.96      0.92     12011
not_sarcastic       0.95      0.83      0.89      9356

    micro avg       0.91      0.91      0.91     21367
    macro avg       0.91      0.90      0.90     21367
 weighted avg       0.91      0.91      0.91     21367



**Prediction on Test Set**

In [90]:
print(
    metrics.classification_report(
        test_Y,
        model.predict(test_X),
        target_names = ['is_sarcastic','not_sarcastic'],
    )
)

               precision    recall  f1-score   support

 is_sarcastic       0.79      0.90      0.84      2974
not_sarcastic       0.85      0.69      0.76      2368

    micro avg       0.81      0.81      0.81      5342
    macro avg       0.82      0.80      0.80      5342
 weighted avg       0.81      0.81      0.81      5342



**END**

**Some Keras Stuff I have been playing around and trying to figure out**


In [81]:
dataset = data_en
X=[]
Y=[]
for i in range(len(dataset)):
    X.append(data.iloc[i,0])
    Y.append([data.iloc[i,1]])
y = np.array(Y)

In [44]:
Xs_train, Xs_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state =42, shuffle=True)

In [14]:
import keras

Using TensorFlow backend.


In [15]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [16]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(Xs_train)

X_train = tokenizer.texts_to_sequences(Xs_train)
X_test = tokenizer.texts_to_sequences(Xs_test)

vocab_size = len(tokenizer.word_index) + 1

In [17]:
print(vocab_size)

20097


In [18]:
print(Xs_train[0])
print(X_train[0])

annoying ad turn man pro whaling
[2271, 288, 255, 3, 767, 11099]


In [19]:
maxlen = 32
X_train = pad_sequences(X_train, padding='post', maxlen = maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen= maxlen)

In [20]:
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Dropout, Flatten, LSTM
from keras.layers import Conv1D,GlobalMaxPool1D,MaxPooling1D

**My Trash Model**

Had no idea what those layers are, might as well try it out

Eeek 50% accuracy

In [21]:
from keras.models import Sequential
from keras import layers

embedding_dim = 128

model = Sequential()
model.add(Dense(64, input_shape=(32,),activation ='relu'))
model.add(Dropout(0.5))
model.add(Dense(32,activation ='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation ='sigmoid'))
model.compile(optimizer = keras.optimizers.Nadam(),
             loss='binary_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


**Some guy's model I stole in Kaggle**

99% accuracy, wow

source:https://www.kaggle.com/vrishabhps/sarcasm-detection-using-keras-and-nltk

In [29]:
from keras.models import Sequential
from keras import layers

embedding_dim = 128

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim,input_length=maxlen))
model.add(layers.LSTM(embedding_dim, return_sequences = True))
model.add(layers.Conv1D(128,3,activation='relu'))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10,activation='relu'))
model.add(layers.Dense(1,activation ='sigmoid'))
model.compile(optimizer = keras.optimizers.Nadam(),
             loss='binary_crossentropy',
              metrics=['accuracy'])

In [31]:

model.fit(X_train,y_train,validation_data=(X_test,y_test), batch_size=10,epochs=5,verbose=2)

Train on 21367 samples, validate on 5342 samples
Epoch 1/5
 - 259s - loss: 0.0136 - acc: 0.9952 - val_loss: 1.2838 - val_acc: 0.7900
Epoch 2/5
 - 246s - loss: 0.0096 - acc: 0.9965 - val_loss: 1.3408 - val_acc: 0.7879
Epoch 3/5
 - 245s - loss: 0.0088 - acc: 0.9971 - val_loss: 1.7028 - val_acc: 0.7782
Epoch 4/5
 - 247s - loss: 0.0060 - acc: 0.9981 - val_loss: 1.4560 - val_acc: 0.7870
Epoch 5/5
 - 253s - loss: 0.0067 - acc: 0.9978 - val_loss: 1.3041 - val_acc: 0.7812
